<a href="https://colab.research.google.com/github/Ivette12345/SMU-hw-Sep_2023/blob/main/charity_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Preprocessing

In [176]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import Adam
import tables  # Import PyTables library for working with HDF5 files
!pip install tables
!pip install keras-tuner
import keras_tuner as kt

# suppress warnings
import warnings
warnings.filterwarnings('ignore')

In [177]:
#  Import and read the charity_data.csv.
import pandas as pd
df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [178]:
df.info()
# No nulls, no missing values but many strings

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34299 entries, 0 to 34298
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   EIN                     34299 non-null  int64 
 1   NAME                    34299 non-null  object
 2   APPLICATION_TYPE        34299 non-null  object
 3   AFFILIATION             34299 non-null  object
 4   CLASSIFICATION          34299 non-null  object
 5   USE_CASE                34299 non-null  object
 6   ORGANIZATION            34299 non-null  object
 7   STATUS                  34299 non-null  int64 
 8   INCOME_AMT              34299 non-null  object
 9   SPECIAL_CONSIDERATIONS  34299 non-null  object
 10  ASK_AMT                 34299 non-null  int64 
 11  IS_SUCCESSFUL           34299 non-null  int64 
dtypes: int64(4), object(8)
memory usage: 3.1+ MB


In [179]:
df.describe()
# ASK_AMT needs to be scaled

,EIN,STATUS,ASK_AMT,IS_SUCCESSFUL
count,3.429900e+04,34299.000000,3.429900e+04,34299.000000
mean,5.191852e+08,0.999854,2.769199e+06,0.532406
std,2.451472e+08,0.012073,8.713045e+07,0.498956
min,1.052060e+07,0.000000,5.000000e+03,0.000000
25%,2.748482e+08,1.000000,5.000000e+03,0.000000
50%,4.656317e+08,1.000000,5.000000e+03,1.000000
75%,7.526117e+08,1.000000,7.742000e+03,1.000000
max,9.960869e+08,1.000000,8.597806e+09,1.000000


In [180]:
df.IS_SUCCESSFUL.value_counts()
#The target is balanced

1    18261
0    16038
Name: IS_SUCCESSFUL, dtype: int64

In [181]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'. My target is IS_SUCCESSFUL. My features are APPLICATION_TYPE, AFFILIATION, CLASSIFICATION,
# USE_CASE, ORGANIZATION, STATUS, INCOME_AMT, SPECIAL_CONSIDERATIONS, ASK_AMT
df2=df.drop(["EIN", "NAME"], axis=1)
df2.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [182]:
# Determine the number of unique values for each column. Application Type and Classification are the only columns we have to bucket because they have over 10 values. Ask_AMT is not categorical.
df2.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [183]:
# Look at APPLICATION_TYPE value counts for binning
counts =df2["APPLICATION_TYPE"].value_counts()
counts

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [184]:
# Choose a cutoff value and create a list of application types to be replaced. Cutoff value 528. Do bucketing to move lower ones into "other".
# use the variable name `application_types_to_replace`
application_types_to_replace = list(counts[counts<500].index)
#application_types_to_replace
# Replace in dataframe
for app in application_types_to_replace:
    df2['APPLICATION_TYPE'] = df2['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
df2['APPLICATION_TYPE'].value_counts()


T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [185]:
# Look at CLASSIFICATION value counts for binning
classification_count=df2["CLASSIFICATION"].value_counts()
classification_count

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [186]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classification_count2 = classification_count[classification_count>1]
classification_count2

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [187]:
# Choose a cutoff value and create a list of classifications to be replaced. Cut off is 1800. Do bucketing to move lower ones into "other."
# use the variable name `classifications_to_replace`
classifications_to_replace = list (classification_count[classification_count<1800].index)
classifications_to_replace

['C7000',
 'C1700',
 'C4000',
 'C5000',
 'C1270',
 'C2700',
 'C2800',
 'C7100',
 'C1300',
 'C1280',
 'C1230',
 'C1400',
 'C7200',
 'C2300',
 'C1240',
 'C8000',
 'C7120',
 'C1500',
 'C1800',
 'C6000',
 'C1250',
 'C8200',
 'C1238',
 'C1278',
 'C1235',
 'C1237',
 'C7210',
 'C2400',
 'C1720',
 'C4100',
 'C1257',
 'C1600',
 'C1260',
 'C2710',
 'C0',
 'C3200',
 'C1234',
 'C1246',
 'C1267',
 'C1256',
 'C2190',
 'C4200',
 'C2600',
 'C5200',
 'C1370',
 'C1248',
 'C6100',
 'C1820',
 'C1900',
 'C1236',
 'C3700',
 'C2570',
 'C1580',
 'C1245',
 'C2500',
 'C1570',
 'C1283',
 'C2380',
 'C1732',
 'C1728',
 'C2170',
 'C4120',
 'C8210',
 'C2561',
 'C4500',
 'C2150']

In [188]:
# Replace in dataframe
for cls in classifications_to_replace:
  df2['CLASSIFICATION'] = df2['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
df2['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [189]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34299 entries, 0 to 34298
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   APPLICATION_TYPE        34299 non-null  object
 1   AFFILIATION             34299 non-null  object
 2   CLASSIFICATION          34299 non-null  object
 3   USE_CASE                34299 non-null  object
 4   ORGANIZATION            34299 non-null  object
 5   STATUS                  34299 non-null  int64 
 6   INCOME_AMT              34299 non-null  object
 7   SPECIAL_CONSIDERATIONS  34299 non-null  object
 8   ASK_AMT                 34299 non-null  int64 
 9   IS_SUCCESSFUL           34299 non-null  int64 
dtypes: int64(3), object(7)
memory usage: 2.6+ MB


In [190]:
# Convert categorical data to numeric with `pd.get_dummies`
df2=pd.get_dummies(df2)
df2.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,1,0


In [191]:
# Split our preprocessed data into our features and target arrays. X is going to be everything except the target. Take X as the input and want to predict y the output.
y=df2["IS_SUCCESSFUL"].values
y
X=df2.drop("IS_SUCCESSFUL", axis=1).values
X

array([[       1,     5000,        0, ...,        0,        1,        0],
       [       1,   108590,        0, ...,        0,        1,        0],
       [       1,     5000,        0, ...,        0,        1,        0],
       ...,
       [       1,     5000,        0, ...,        0,        1,        0],
       [       1,     5000,        0, ...,        0,        1,        0],
       [       1, 36500179,        0, ...,        0,        1,        0]])

In [192]:
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

print(X_train.shape)
print(X_test.shape)

(25724, 43)
(8575, 43)


In [193]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [194]:
# Define the model - the number of input features
number_input_features = len(X_train_scaled[0])

# Set the number of neurons for each hidden layer
hidden_nodes_layer1 = 32
hidden_nodes_layer2 = 64

# Create a Sequential model
nn_model1 = tf.keras.models.Sequential()

# Add the first hidden layer with 32 neurons
nn_model1.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Add the second hidden layer with 64 neurons
nn_model1.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Add the output layer that uses a probability activation function
nn_model1.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the model
nn_model1.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Define the callback to save the model's weights every five epochs
checkpoint_filepath = 'AlphabetSoupCharity.h5'
model_checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    save_freq='epoch',
    period=5
)

# Train the model
nn_model1.fit(X_train_scaled, y_train, epochs=50, batch_size=32, validation_data=(X_test_scaled, y_test), callbacks=[model_checkpoint_callback])

# Evaluate the model
model_loss, model_accuracy = nn_model1.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Number of neurons in each hidden layer: {hidden_nodes_layer1, hidden_nodes_layer2}, Loss: {model_loss}, Accuracy: {model_accuracy}")

# Save the model to an HDF5 file
nn_model1.save('AlphabetSoupCharity.h5')

# Check the structure of the model
nn_model1.summary()

Epoch 1/50
804/804 [==============================] - 4s 4ms/step - loss: 0.5773 - accuracy: 0.7161 - val_loss: 0.5629 - val_accuracy: 0.7233
Epoch 2/50
804/804 [==============================] - 4s 4ms/step - loss: 0.5569 - accuracy: 0.7314 - val_loss: 0.5620 - val_accuracy: 0.7234
Epoch 3/50
804/804 [==============================] - 7s 8ms/step - loss: 0.5543 - accuracy: 0.7321 - val_loss: 0.5573 - val_accuracy: 0.7249
Epoch 4/50
804/804 [==============================] - 5s 6ms/step - loss: 0.5522 - accuracy: 0.7327 - val_loss: 0.5552 - val_accuracy: 0.7242
Epoch 5/50
804/804 [==============================] - 3s 4ms/step - loss: 0.5502 - accuracy: 0.7319 - val_loss: 0.5599 - val_accuracy: 0.7228
Epoch 6/50
804/804 [==============================] - 3s 3ms/step - loss: 0.5500 - accuracy: 0.7331 - val_loss: 0.5553 - val_accuracy: 0.7245
Epoch 7/50
804/804 [==============================] - 2s 3ms/step - loss: 0.5482 - accuracy: 0.7338 - val_loss: 0.5560 - val_accuracy: 0.7249
Epoch 

In [195]:
# Define the number of input features
number_input_features = len(X_train_scaled[0])

# Define the number of neurons in each hidden layer
hidden_nodes_layer1 = 32
hidden_nodes_layer2 = 64
hidden_nodes_layer3 = 64

# Create a Sequential model
nn_model2 = tf.keras.models.Sequential()

# Add the first hidden layer with 32 neurons
nn_model2.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Add the second hidden layer with 64 neurons
nn_model2.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Add the third hidden layer with 64 neurons
nn_model2.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Add the output layer that uses a probability activation function
nn_model2.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the model with a specific learning rate
opt = Adam(learning_rate=0.001)  # Specify the learning rate here
nn_model2.compile(loss="binary_crossentropy", optimizer=opt, metrics=["accuracy"])

# Define the callback to save the model's weights every five epochs
checkpoint_filepath = 'AlphabetSoupCharity.h5'
model_checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    save_freq='epoch',
    period=5
)

# Train the model
nn_model2.fit(X_train_scaled, y_train, epochs=50, batch_size=32, validation_data=(X_test_scaled, y_test), callbacks=[model_checkpoint_callback])

# Evaluate the model
model_loss, model_accuracy = nn_model2.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Number of neurons in each hidden layer: {hidden_nodes_layer1, hidden_nodes_layer2, hidden_nodes_layer3}, Loss: {model_loss}, Accuracy: {model_accuracy}")

# Check the structure of the model
nn_model2.summary()

Epoch 1/50
804/804 [==============================] - 4s 3ms/step - loss: 0.5746 - accuracy: 0.7203 - val_loss: 0.5602 - val_accuracy: 0.7222
Epoch 2/50
804/804 [==============================] - 3s 3ms/step - loss: 0.5555 - accuracy: 0.7303 - val_loss: 0.5577 - val_accuracy: 0.7241
Epoch 3/50
804/804 [==============================] - 2s 3ms/step - loss: 0.5524 - accuracy: 0.7312 - val_loss: 0.5572 - val_accuracy: 0.7263
Epoch 4/50
804/804 [==============================] - 2s 3ms/step - loss: 0.5511 - accuracy: 0.7327 - val_loss: 0.5532 - val_accuracy: 0.7258
Epoch 5/50
804/804 [==============================] - 2s 3ms/step - loss: 0.5492 - accuracy: 0.7333 - val_loss: 0.5559 - val_accuracy: 0.7273
Epoch 6/50
804/804 [==============================] - 2s 3ms/step - loss: 0.5482 - accuracy: 0.7342 - val_loss: 0.5538 - val_accuracy: 0.7273
Epoch 7/50
804/804 [==============================] - 3s 3ms/step - loss: 0.5476 - accuracy: 0.7345 - val_loss: 0.5536 - val_accuracy: 0.7272
Epoch 

In [196]:
# Define the number of input features
number_input_features = len(X_train_scaled[0])

# Set the number of neurons for each hidden layer
hidden_nodes_layer1 = 64
hidden_nodes_layer2 = 32
hidden_nodes_layer3 = 16
hidden_nodes_layer4 = 8

# Create a Sequential model
nn_model3 = tf.keras.models.Sequential()

# Add the first hidden layer with 64 neurons
nn_model3.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Add the second hidden layer with 32 neurons
nn_model3.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Add the third hidden layer with 16 neurons
nn_model3.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Add the fourth hidden layer with 8 neurons
nn_model3.add(tf.keras.layers.Dense(units=hidden_nodes_layer4, activation="relu"))

# Add the output layer that uses a probability activation function
nn_model3.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the model with a specific learning rate
opt = Adam(learning_rate=0.001)  # Specify the learning rate here
nn_model3.compile(loss="binary_crossentropy", optimizer=opt, metrics=["accuracy"])

# Define the callback to save the model's weights every five epochs
checkpoint_filepath = 'AlphabetSoupCharity.h5'
model_checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    save_freq='epoch',
    period=5
)

# Train the model
nn_model3.fit(X_train_scaled, y_train, epochs=100, batch_size=32, validation_data=(X_test_scaled, y_test), callbacks=[model_checkpoint_callback])

# Evaluate the model
model_loss, model_accuracy = nn_model3.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Number of neurons in each hidden layer: {hidden_nodes_layer1, hidden_nodes_layer2, hidden_nodes_layer3, hidden_nodes_layer4}, Loss: {model_loss}, Accuracy: {model_accuracy}")

# Check the structure of the model
nn_model3.summary()

Epoch 1/100
804/804 [==============================] - 4s 3ms/step - loss: 0.5706 - accuracy: 0.7215 - val_loss: 0.5608 - val_accuracy: 0.7238
Epoch 2/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5546 - accuracy: 0.7302 - val_loss: 0.5589 - val_accuracy: 0.7256
Epoch 3/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5514 - accuracy: 0.7315 - val_loss: 0.5555 - val_accuracy: 0.7252
Epoch 4/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5500 - accuracy: 0.7331 - val_loss: 0.5538 - val_accuracy: 0.7259
Epoch 5/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5486 - accuracy: 0.7332 - val_loss: 0.5530 - val_accuracy: 0.7245
Epoch 6/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5481 - accuracy: 0.7339 - val_loss: 0.5549 - val_accuracy: 0.7257
Epoch 7/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5473 - accuracy: 0.7341 - val_loss: 0.5538 - val_accuracy: 0.7271

In [197]:
# Define the number of input features
number_input_features = len(X_train_scaled[0])

# Set the number of neurons for each hidden layer
hidden_nodes_layer1 = 64
hidden_nodes_layer2 = 128
hidden_nodes_layer3 = 256
hidden_nodes_layer4 = 512

# Create a Sequential model
nn_model4 = tf.keras.models.Sequential()

# Add the first hidden layer with 64 neurons
nn_model4.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Add the second hidden layer with 128 neurons
nn_model4.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Add the third hidden layer with 256 neurons
nn_model4.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Add the fourth hidden layer with 512 neurons
nn_model4.add(tf.keras.layers.Dense(units=hidden_nodes_layer4, activation="tanh"))

# Add the output layer that uses a probability activation function
nn_model4.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the model with a specific learning rate
opt = Adam(learning_rate=0.001)  # Specify the learning rate here
nn_model4.compile(loss="binary_crossentropy", optimizer=opt, metrics=["accuracy"])

# Define the callback to save the model's weights every five epochs
checkpoint_filepath = 'AlphabetSoupCharity.h5'
model_checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    save_freq='epoch',
    period=5
)

# Train the model
nn_model4.fit(X_train_scaled, y_train, epochs=150, batch_size=32, validation_data=(X_test_scaled, y_test), callbacks=[model_checkpoint_callback])

# Evaluate the model
model_loss, model_accuracy = nn_model4.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Number of neurons in each hidden layer: {hidden_nodes_layer1, hidden_nodes_layer2, hidden_nodes_layer3, hidden_nodes_layer4}, Loss: {model_loss}, Accuracy: {model_accuracy}")

# Save the model to an HDF5 file
nn_model4.save('AlphabetSoupCharity_Optimization.h5')

# Check the structure of the model
nn_model4.summary()

Epoch 1/150
804/804 [==============================] - 6s 6ms/step - loss: 0.5719 - accuracy: 0.7210 - val_loss: 0.5633 - val_accuracy: 0.7262
Epoch 2/150
804/804 [==============================] - 5s 7ms/step - loss: 0.5568 - accuracy: 0.7303 - val_loss: 0.5562 - val_accuracy: 0.7252
Epoch 3/150
804/804 [==============================] - 5s 6ms/step - loss: 0.5537 - accuracy: 0.7313 - val_loss: 0.5580 - val_accuracy: 0.7278
Epoch 4/150
804/804 [==============================] - 6s 7ms/step - loss: 0.5520 - accuracy: 0.7311 - val_loss: 0.5528 - val_accuracy: 0.7272
Epoch 5/150
804/804 [==============================] - 5s 6ms/step - loss: 0.5507 - accuracy: 0.7318 - val_loss: 0.5561 - val_accuracy: 0.7272
Epoch 6/150
804/804 [==============================] - 6s 7ms/step - loss: 0.5489 - accuracy: 0.7335 - val_loss: 0.5542 - val_accuracy: 0.7279
Epoch 7/150
804/804 [==============================] - 5s 6ms/step - loss: 0.5492 - accuracy: 0.7349 - val_loss: 0.5514 - val_accuracy: 0.7269

In [198]:
# Export our model to HDF5 file
#from google.colab import files
#nn_model4.save('AlphabetSoupCharity_Optimization.h5')
#files.download('AlphabetSoupCharity_Optimization.h5')
